In [ ]:
# Install packages that aren't available by default
# You probably don't need this section if you run on your own computer
%pip install shapely --no-binary shapely --no-input --force-reinstall
%pip install "cartopy<0.20" # Version 0.20 currently has version conflict
%pip install proplot

In [ ]:
import os
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from glob import glob
import proplot

# Suppress selected warnings; these are for developers to handle, not us
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [ ]:
# If you are running on Google Colab, need to mount Google Drive, where the model output is stored 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the directories for control and experimental simulations
dir_root = '/content/drive/MyDrive/MET5607-Arctic-change/'
dir_control = dir_root+'control/OutputDir/'
dir_experiment = dir_root+'2050-seaice/OutputDir/'
# dir_experiment = dir_root+'2050-Tq/OutputDir/'

# Check that directories exist
try:
  for dir in [dir_control,dir_experiment]:
    assert os.path.isdir(dir), "Directory not found {:s}".format(dir)
except Exception as e:
  print(e)
  print('If you are using Google Colab, you need to create a shortcut to the shared '+\
        'directory in your Google Drive or copy the files to your Google Drive')

In [ ]:
# Locate all files for each simulation
files_control = []
files_experiment = []
for dir,files in zip([dir_control,dir_experiment],[files_control,files_experiment]):
  files.extend(glob(dir+'GEOSChem.StateMet.2015*.nc4'))
  files.extend(glob(dir+'GEOSChem.SpeciesConc.2015*.nc4'))
  files.extend(glob(dir+'GEOSChem.ConcAfterChem.2015*.nc4'))

  # Also open some or all the following as needed for constructing a budget
  # It may take a long time to open all at once
  #files.extend(glob(dir+'GEOSChem.ProdLoss.2015*.nc4'))
  #files.extend(glob(dir+'GEOSChem.DryDep.2015*.nc4'))
  #files.extend(glob(dir+'GEOSChem.Budget.2015*.nc4'))
  #files.extend(glob(dir+'HEMCO_diagnostics.2015*.nc'))


# Minimal sanity check that we found the right files
assert len(files_control) > 0, 'No files found for control simulation'
assert len(files_control) % 12 == 0, 'Number of files should be a multiple of 12, one for each month'
assert len(files_control) == len(files_experiment), 'Number of files should be same for control and experiment simulations!'

In [ ]:
# Open datasets
control = xr.open_mfdataset(files_control)
expr    = xr.open_mfdataset(files_experiment)

# Level is approximately pressure, hPa
control['lev'] = control['lev'] * control['P0'].isel(time=0)
expr['lev']    = expr['lev']    * expr['P0'].isel(time=0)

In [ ]:
for ds in [control, expr]:
  # Rename HO2 for consistency with other species variables
  ds['SpeciesConc_HO2'] = ds['HO2concAfterChem']
  # Rename OH and convert molec/cm3 -> mol/mol
  ds['SpeciesConc_OH']  = ( ds['OHconcAfterChem'] / 
                                (ds['Met_AIRDEN'] / 0.02897 * 6.02e23 / 1e6 ) 
                                ).assign_attrs( dict(units = 'mol mol-1 dry') )

In [ ]:
# Define families
# If applicable, it would make sense to define chemical families here before annual means are calculated
for ds in [control, expr]:
  ds['SpeciesConc_HOx'] = (ds['SpeciesConc_OH'] + ds['SpeciesConc_HO2']) #/ (1e-9).assign_attrs(dict(units))


In [ ]:
# Days in each month, used for weighting monthly values for annual mean
dpm = xr.DataArray( [31,28,31,30,31,30,31,31,30,31,30,31], coords=control.time.coords )

# Annual mean
control_annual = control.weighted(dpm).mean(dim='time')
expr_annual    = expr.weighted(dpm).mean(dim='time')

In [ ]:
# Days in each month, used for weighting monthly values for annual mean
dpm = xr.DataArray( [31,28,31,30,31,30,31,31,30,31,30,31], coords=control.time.coords )

# Annual mean
control_annual = control.weighted(dpm).mean(dim='time')
expr_annual    = expr.weighted(dpm).mean(dim='time')

In [ ]:
# Define some map projections that we will use

# Global maps. EqualEarth is excellent, but not available in Colab, so use Robinson
# global_proj = ccrs.EqualEarth()
global_proj = ccrs.Robinson()
global_extents = [-180,180,-90,90]

# Arctic maps, use stereographic
arctic_proj = ccrs.NorthPolarStereo()
arctic_extents = [-180,180,65,90]

In [ ]:
# Plot annual mean 2m air temperature
control_annual['Met_TS'].plot()

In [ ]:
# Repeat annual mean air temperature, now as a map with coastlines
# Plot annual mean 2m air temperature
p = control_annual['Met_TS'].plot(
    subplot_kws=dict(projection=global_proj),
    transform=ccrs.PlateCarree()
    )
p.axes.coastlines()
p.axes.gridlines()

In [ ]:
# Plot January 2m air temperature 
# Indices start at 0, so 0=Jan, 1=Feb, ... 11=Dec
p = control['Met_TS'].isel(time=0).plot(
    subplot_kws=dict(projection=global_proj),
    transform=ccrs.PlateCarree()
    )
p.axes.coastlines()

In [ ]:
# Plot annual, zonal mean temperature
# Use yincrease=False so that y axis shows decreasing pressure with height
control_annual['Met_T'].mean(dim='lon').\
  plot( yincrease=False )

In [ ]:
# Plot zonal mean temperature, January
control['Met_T'].isel(time=0).\
  mean(dim='lon').plot(
      yincrease=False,
      levels=15
      )

In [ ]:
# Plot 2m air temperature for all 12 months
p = control['Met_TS'].plot(
    col='time', col_wrap=4,
    subplot_kws=dict(projection=global_proj),
    transform=ccrs.PlateCarree()
    )

for ax in p.axes.flat:
  ax.coastlines()

In [ ]:
# Plot 2m air temperature for all 12 months, Arctic map
p = control['Met_TS'].plot(
    col='time', col_wrap=4,
    subplot_kws=dict(projection=arctic_proj),
    transform=ccrs.PlateCarree(),
    vmin=230,vmax=290
    )

for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )
  # ax.gridlines()

In [ ]:
# Define the HOx family, Concentration Control
control['SpeciesConc_HOx'] = control['SpeciesConc_HO2'] + control['SpeciesConc_OH']
#expr['SpeciesConc_HOx'] = expr['SpeciesConc_HO2'] + expr['SpeciesConc_OH']


# Plot it
p = control['SpeciesConc_HOx'].isel(time=4,lev=5).\
  plot(
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    vmax=1.0e-11
  )
p.axes.coastlines()
p.axes.set_extent( arctic_extents, ccrs.PlateCarree() )

In [ ]:
p = control['SpeciesConc_HOx'].isel(lev=5).plot(
    col='time', col_wrap=4,
    subplot_kws=dict(projection=arctic_proj),
    transform=ccrs.PlateCarree()
    )

for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )
  # ax.gridlines()

In [ ]:
# Define the HOx family, Concentration Experiment
#control['SpeciesConc_HOx'] = control['SpeciesConc_HO2'] + control['SpeciesConc_OH']
expr['SpeciesConc_HOx'] = expr['SpeciesConc_HO2'] + expr['SpeciesConc_OH']


# Plot it
p = expr['SpeciesConc_HOx'].isel(time=4,lev=0).\
  plot(
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    vmax=1.0e-11
  )
p.axes.coastlines()
p.axes.set_extent( arctic_extents, ccrs.PlateCarree() )

In [ ]:
p = control['SpeciesConc_HOx'].isel(lev=5).plot(
    col='time', col_wrap=4,
    subplot_kws=dict(projection=arctic_proj),
    transform=ccrs.PlateCarree()
    )

for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )
  # ax.gridlines()

In [ ]:
# Define the HOx family
#control['SpeciesConc_HOx'] = control['SpeciesConc_HO2'] + control['SpeciesConc_OH']
#expr['SpeciesConc_HOx'] = expr['SpeciesConc_HO2'] + expr['SpeciesConc_OH']


# Plot it
p = diff['SpeciesConc_HOx'].isel(time=4,lev=0).\
  plot(
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    #vmax=1.0e-11
  )
p.axes.coastlines()
p.axes.set_extent( arctic_extents, ccrs.PlateCarree() )

In [ ]:
# Sea ice concentrations
p = control['Met_FRSEAICE'].isel(time=[0,6]).plot(
    col='time',
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    vmin=0, vmax=1, cmap='blues'
    )
for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )

In [ ]:
# Sea ice concentrations
p = expr['Met_FRSEAICE'].isel(time=[0,6]).plot(
    col='time',
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    vmin=0, vmax=1, cmap='blues'
    )
for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )

In [ ]:
# Sea ice concentrations differences b/t control and expr
diff = control['Met_FRSEAICE'] - expr['Met_FRSEAICE']
p = diff.isel(time=[0,6]).plot(
    col='time',
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    vmin=0, vmax=1, cmap='blues'
    )
for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )

In [ ]:
# Change in sea ice concentration between control and experiment simulations
# This should be non-zero for 2050-seaice and zero for 2050-Tq
#diff = control['Met_FRSEAICE'] - expr['Met_FRSEAICE']
#diff = control['SpeciesConc_HOx'] - expr['SpeciesConc_HOx']
p = diff['SpeciesConc_HOx'].isel(lev=0).plot(
    col='time', col_wrap=4,
    subplot_kws=dict(projection=arctic_proj),
    transform=ccrs.PlateCarree()
    )

for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )
  # ax.gridlines()

In [ ]:
# Change in sea ice concentration between control and experiment simulations
# This should be non-zero for 2050-seaice and zero for 2050-Tq
p = diff['Met_FRSEAICE'].plot(
    col='time', col_wrap=4,
    subplot_kws=dict(projection=arctic_proj),
    transform=ccrs.PlateCarree()
    )

for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )
  # ax.gridlines()

In [ ]:
# Temperature change in lowest model level
# 2M Monthly temperature diff b/t control and expr
p = diff['Met_TS'].plot(
    col='time', col_wrap=4,
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    cbar_kwargs={'label':r'$\Delta$ T, K'}
    )

p.set_titles(template='{value}')

for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )
  # ax.gridlines()

In [ ]:
# Temperature change in lowest model level
# 2M Monthly specific humidity diff b/t control and expr
p = diff['Met_QV2M'].plot(
    col='time', col_wrap=4,
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    cbar_kwargs={'label':r'$\Delta$ q, kg/kg'},
    vmax = 10e-4
    )

p.set_titles(template='{value}')

for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )
  # ax.gridlines()

In [ ]:
# Annual, zonal mean temperature change
p = diff_annual['Met_T'].mean(dim='lon').plot.pcolormesh(
    yincrease=False,
    xlim=[60,90],
    vmin=0, vmax=2, cmap='reds',
    cbar_kwargs={'label':r'$\Delta$ T, K'}
    )
p.axes.set_xlabel('Latitude')
p.axes.set_ylabel('Pressure, hPa')
plt.tight_layout()

In [ ]:
# Annual, zonal mean change in specific humidity
p = expr_annual['SpeciesConc_HOx'].mean(dim='lon').plot.pcolormesh(
    yincrease=False,
    xlim=[60,90],
    cmap='greens',
    cbar_kwargs={'label':r'Zonal HOx Concentration'}, 
    vmax = 4e-12
    )
p.axes.set_xlabel('Latitude')
p.axes.set_ylabel('Pressure, hPa')
plt.tight_layout()

In [ ]:
# Annual, zonal mean change in specific humidity
p = diff['SpeciesConc_HOx'].mean(dim='lon').plot.pcolormesh(
    col='time', col_wrap=4,
    yincrease=False,
    xlim=[60,90],
    cmap='greens',
    cbar_kwargs={'label':r'Zonal HOx Concentration'}, 
    vmax = 4e-13
    )
p.axes.set_xlabel('Latitude')
p.axes.set_ylabel('Pressure, hPa')
plt.tight_layout()

In [ ]:
# Annual mean change in air temperature, lowest model level
p = diff_annual['Met_T'].isel(lev=0).plot(
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    #vmin=0,
    #vmax=3,
    cmap='reds', 
    cbar_kwargs={'label':r'$\Delta$ T, K'}
    )
p.axes.coastlines()
p.axes.set_extent( arctic_extents, ccrs.PlateCarree() )

In [ ]:
# Annual mean change in specific humidity, lowest model level
p = diff_annual['Met_SPHU'].isel(lev=0).plot(
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    #vmin=0, 
    #vmax=0.5, 
    cmap='greens', 
    cbar_kwargs={'label':r'$\Delta$ q, kg/kg'}
    )
p.axes.coastlines()
p.axes.set_extent( arctic_extents, ccrs.PlateCarree() )

In [ ]:
# Compute annual mean burdens for HOx Arctic Troposphere

# List of variables that are species concentrations
species_vars = [varname for varname in list(control_annual.data_vars) if varname.startswith('SpeciesConc_')]

# Burden in the Arctic Troposphere, all species
# SpeciesConc - mol/mol
# AD - kg
# FracOfTimeInTrop - unitless [0,1]
# control_burden_annual - moles
control_burden_annual = ( control_annual[species_vars] * 
                      control_annual['Met_AD'] / 0.02897 * 
                      control_annual['FracOfTimeInTrop'] 
                     ).sel(lat=slice(66,90)).sum()
expr_burden_annual = ( expr_annual[species_vars] * 
                      expr_annual['Met_AD'] / 0.02897 * 
                      expr_annual['FracOfTimeInTrop'] 
                     ).sel(lat=slice(66,90)).sum()

# Choose species or family to print
spc = 'HOx'

# Print the burden
print('Arctic Troposphere {:s}'.format(spc))
print('Burden {:g} mol'.format( control_burden_annual['SpeciesConc_'+spc].values ) )
print('Arctic Troposphere {:s}'.format(spc))
print('Burden {:g} mol'.format( expr_burden_annual['SpeciesConc_'+spc].values ) )

In [ ]:
# Compute annual production and loss for HOx Arctic Troposphere

# List of variables that are species concentrations
PL_vars = [varname for varname in list(control_annual.data_vars) if (varname.startswith('Prod_') or varname.startswith('Loss_'))]

# Production & loss in the Arctic Troposphere, all species
# Prod, Loss - molec/cm3/s
# AIRVOL - m3
# FracOfTimeInTrop - unitless [0,1]
# control_pl_annual - moles / yr
control_pl_annual = ( control_annual[PL_vars] * 1e6 *
                      control_annual['Met_AIRVOL']  * 
                      control_annual['FracOfTimeInTrop'] * 365 * 86400 / 6.02e23
                     ).sel(lat=slice(66,90)).sum()

expr_pl_annual = ( expr_annual[PL_vars] * 1e6 *
                      expr_annual['Met_AIRVOL']  * 
                      expr_annual['FracOfTimeInTrop'] * 365 * 86400 / 6.02e23
                     ).sel(lat=slice(66,90)).sum()

# Choose species or family to print
spc = 'HOx'

# Print the burden
print('Arctic Troposphere {:s} Control'.format(spc))
print('Prod {:g} mol/yr'.format( control_pl_annual['Prod_'+spc].values ) )
print('Loss {:g} mol/yr'.format( control_pl_annual['Loss_'+spc].values ) )
print('Arctic Troposphere {:s} Experiment'.format(spc))
print('Prod {:g} mol/yr'.format( expr_pl_annual['Prod_'+spc].values ) )
print('Loss {:g} mol/yr'.format( expr_pl_annual['Loss_'+spc].values ) )

In [ ]:
# Compute annual mean burdens for OH Arctic Troposphere

# List of variables that are species concentrations
species_vars = [varname for varname in list(control_annual.data_vars) if varname.startswith('SpeciesConc_')]

# Burden in the Arctic Troposphere, all species
# SpeciesConc - mol/mol
# AD - kg
# FracOfTimeInTrop - unitless [0,1]
# control_burden_annual - moles
control_burden_annual = ( control_annual[species_vars] * 
                      control_annual['Met_AD'] / 0.02897 * 
                      control_annual['FracOfTimeInTrop'] 
                     ).sel(lat=slice(66,90)).sum()
expr_burden_annual = ( expr_annual[species_vars] * 
                      expr_annual['Met_AD'] / 0.02897 * 
                      expr_annual['FracOfTimeInTrop'] 
                     ).sel(lat=slice(66,90)).sum()

# Choose species or family to print
spc = 'OH'

# Print the burden
print('Arctic Troposphere {:s}'.format(spc))
print('Burden {:g} mol'.format( control_burden_annual['SpeciesConc_'+spc].values ) )
print('Arctic Troposphere {:s}'.format(spc))
print('Burden {:g} mol'.format( expr_burden_annual['SpeciesConc_'+spc].values ) )

In [ ]:
# Compute annual production and loss for OH Arctic Troposphere

# List of variables that are species concentrations
PL_vars = [varname for varname in list(control_annual.data_vars) if (varname.startswith('Prod_') or varname.startswith('Loss_'))]

# Production & loss in the Arctic Troposphere, all species
# Prod, Loss - molec/cm3/s
# AIRVOL - m3
# FracOfTimeInTrop - unitless [0,1]
# control_pl_annual - moles / yr
control_pl_annual = ( control_annual[PL_vars] * 1e6 *
                      control_annual['Met_AIRVOL']  * 
                      control_annual['FracOfTimeInTrop'] * 365 * 86400 / 6.02e23
                     ).sel(lat=slice(66,90)).sum()

expr_pl_annual = ( expr_annual[PL_vars] * 1e6 *
                      expr_annual['Met_AIRVOL']  * 
                      expr_annual['FracOfTimeInTrop'] * 365 * 86400 / 6.02e23
                     ).sel(lat=slice(66,90)).sum()

# Choose species or family to print
spc = 'OH'

# Print the burden
print('Arctic Troposphere {:s} Control'.format(spc))
print('Prod {:g} mol/yr'.format( control_pl_annual['Prod_'+spc].values ) )
print('Loss {:g} mol/yr'.format( control_pl_annual['Loss_'+spc].values ) )
print('Arctic Troposphere {:s} Experiment'.format(spc))
print('Prod {:g} mol/yr'.format( expr_pl_annual['Prod_'+spc].values ) )
print('Loss {:g} mol/yr'.format( expr_pl_annual['Loss_'+spc].values ) )

In [ ]:
# Compute annual production and loss for OH Arctic Troposphere

# List of variables that are species concentrations
PL_vars = [varname for varname in list(control_annual.data_vars) if (varname.startswith('Prod_') or varname.startswith('Loss_'))]

# Production & loss in the Arctic Troposphere, all species
# Prod, Loss - molec/cm3/s
# AIRVOL - m3
# FracOfTimeInTrop - unitless [0,1]
# control_pl_annual - moles / yr
control_pl_annual = ( control_annual[PL_vars] * 1e6 *
                      control_annual['Met_AIRVOL']  * 
                      control_annual['FracOfTimeInTrop'] * 365 * 86400 / 6.02e23
                     ).sel(lat=slice(66,90)).sum()

expr_pl_annual = ( expr_annual[PL_vars] * 1e6 *
                      expr_annual['Met_AIRVOL']  * 
                      expr_annual['FracOfTimeInTrop'] * 365 * 86400 / 6.02e23
                     ).sel(lat=slice(66,90)).sum()

# Choose species or family to print
spc = 'OHpri'

# Print the burden
print('Arctic Troposphere {:s} Control'.format(spc))
print('Prod {:g} mol/yr'.format( control_pl_annual['Prod_'+spc].values ) )
print('Arctic Troposphere {:s} Experiment'.format(spc))
print('Prod {:g} mol/yr'.format( expr_pl_annual['Prod_'+spc].values ) )

In [ ]:
# Print a full budget for HOx Family
spc = 'HOx'
spc2 = 'OH'
spc3 = 'OHpri'

# Format for each line
budgetline = '{:15s} {:8g} {:8s}'


#HOx
print('Arctic Troposphere {:s}, Control'.format(spc))
print(budgetline.format( 'Burden',       control_burden_annual['SpeciesConc_{:s}'.format(spc)].values,        'mol'))

print(budgetline.format( 'Chem Prod',    control_pl_annual['Prod_{:s}'.format(spc)].values,                   'mol/yr'))
print(budgetline.format( 'Chem Loss',    control_pl_annual['Loss_{:s}'.format(spc)].values,                   'mol/yr'))
print()
print('Arctic Troposphere {:s}, Experiment'.format(spc))
print(budgetline.format( 'Burden',       expr_burden_annual['SpeciesConc_{:s}'.format(spc)].values,        'mol'))

print(budgetline.format( 'Chem Prod',    expr_pl_annual['Prod_{:s}'.format(spc)].values,                   'mol/yr'))
print(budgetline.format( 'Chem Loss',    expr_pl_annual['Loss_{:s}'.format(spc)].values,                   'mol/yr'))
print()


#OH
print('Arctic Troposphere {:s}, Control'.format(spc2))
print(budgetline.format( 'Burden',       control_burden_annual['SpeciesConc_{:s}'.format(spc2)].values,        'mol'))

print(budgetline.format( 'Chem Prod',    control_pl_annual['Prod_{:s}'.format(spc2)].values,                   'mol/yr'))
print(budgetline.format( 'Chem Loss',    control_pl_annual['Loss_{:s}'.format(spc2)].values,                   'mol/yr'))
print()
print('Arctic Troposphere {:s}, Experiment'.format(spc2))
print(budgetline.format( 'Burden',       expr_burden_annual['SpeciesConc_{:s}'.format(spc2)].values,        'mol'))

print(budgetline.format( 'Chem Prod',    expr_pl_annual['Prod_{:s}'.format(spc2)].values,                   'mol/yr'))
print(budgetline.format( 'Chem Loss',    expr_pl_annual['Loss_{:s}'.format(spc2)].values,                   'mol/yr'))


print()
print('Arctic Troposphere {:s}, Control'.format(spc3))
print(budgetline.format( 'Chem Prod',    control_pl_annual['Prod_{:s}'.format(spc3)].values,                   'mol/yr'))
print()
print('Arctic Troposphere {:s}, Experiment'.format(spc3))
print(budgetline.format( 'Chem Prod',    expr_pl_annual['Prod_{:s}'.format(spc3)].values,                   'mol/yr'))


In [ ]:
#Burden HOx / Loss of OH 

#Partial lifetime HOx due to OH loss, Experiment
CPP = 1.40908e+07 / 2.01261e+12

#Partial lifetime HOx due to OH loss, Experiment
EPP = 1.41163e+07 / 2.01524e+12

#Chain Length, Control
CPP2 = 1.70839e+12 / 4.68559e+11

#Chain Length, Experiment
EPP2 = 1.70917e+12 / 4.68909e+11

#Lifetime, Control
CPP3 = 1.40908e+07 / 1.70839e+12

#Lifetime, Experiment
EPP3 = 1.41163e+07 / 1.70916e+12

print('Partial Lifetime of HOx due to OH loss, Control')
print(CPP)
print()
print('Partial Lifetime of HOx due to OH loss, Experiment')
print(EPP)
print()
print('Chain Length, Control')
print(CPP2)
print()
print('Chain Length, Experiment')
print(EPP2)
print()
print('Lifetime HOx, Control')
print(CPP3, 'year')
print()
print('Lifetime HOx, Experiment')
print(EPP3 , 'year')
print()
print('Lifetime HOx, Control')
print(CPP3*365, 'day')
print()
print('Lifetime HOx, Experiment')
print(EPP3*365, 'day')
print()
print('Lifetime HOx, Control')
print(CPP3*(365*24), 'hour')
print()
print('Lifetime HOx, Experiment')
print(EPP3*(365*24), 'hour')